# 4. Prepare the Data for Machine Learning Algorithms

In [70]:
import pandas as pd
import numpy as np
housing = pd.read_csv("housing_clean.csv")  # 불러오기

In [30]:
import pickle

# 저장된 strat_train_set 불러오기
with open('strat_train_set.pkl', 'rb') as f:
    strat_train_set = pickle.load(f)

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()


머신러닝 알고리즘을 위한 데이터 준비

이를 수동으로 처리하는 대신 함수를 작성하는 이유
- 어떤 데이터셋이든 이러한 변환을 쉽게 재현할 수 있음(예: 다음에 새로운 데이터셋을 받을 때)
- 앞으로 프로젝트를 진행하면서 재사용할 수 있는 변환 함수들의 라이브러리를 점차 구축하게 됨
- 이러한 함수들을 실제 시스템에서 사용할 수 있으며 새로운 데이터를 알고리즘에 전달하기 전에 변환하는 데 사용할 수 있음
- 다양한 변환을 쉽게 시도해보고 어떤 조합이 가장 잘 작동하는지 확인 가능

먼저 깨끗한 훈련 세트로 되돌림(strat_train_set을 다시 한 번 복사)  
예측 변수와 레이블을 분리(예측 변수와 타깃 값에 반드시 같은 변환을 적용할 필요는 없기 때문)
>`drop()`: 데이터의 복사본을 만들며 strat_train_set에는 영향 X

In [31]:
housing = strat_train_set.drop("median_house_value", axis=1) # 훈련 세트에서 레이블을 제거
# axis=1은 컬럼 제거, axis=0이면 행 제거
housing_labels = strat_train_set["median_house_value"].copy()

## Data Cleaning(데이터 정제)

In [32]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,NaN,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,NaN,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,NaN,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,NaN,1145.0,480.0,6.3580,NEAR OCEAN


대부분의 머신러닝 알고리즘은 누락된 특성이 있는 데이터를 처리할 수 없음
-> `total_bedrooms` 속성에 일부 누락된 값이 있으므로 해결해야 함

3가지 옵션
1. 해당 구역(districts) 제거 -> `dropna()`
2. 전체 속성 제거 -> `drop()`
3. 누락된 값을 어떤 값(예: 0, 평균, 중간값 등)으로 대체 -> `fillna()`


In [33]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # 옵션 1 null 값을 포함한 행 삭제 

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


In [34]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # 옵션 2 열 자체를 삭제  

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,1145.0,480.0,6.3580,NEAR OCEAN


In [35]:
median = housing["total_bedrooms"].median()  # 옵션 3 열의 median을 계산하여, null 값을 그 중앙값으로 채우기  
sample_incomplete_rows["total_bedrooms"] = sample_incomplete_rows["total_bedrooms"].fillna(median)

In [36]:
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580,NEAR OCEAN


옵션 3은 반드시 훈련 세트에서 중간값을 계산하고, 그 값을 사용해 훈련 세트의 누락된 값을 채워야 함  
-> 계산한 중간값은 **반드시 저장**
>나중에 시스템을 평가할 때 테스트 세트의 누락값을 채우는 데 사용하고,   
실제 운영 환경에서도 새로운 데이터의 누락값을 대체하는 데 필요하기 때문

누락값 처리를 위한 편리한 클래스인 `SimpleImputer`(Scikit-Learn)

먼저 각 속성의 누락된 값을 해당 속성의 중간값으로 대체하도록 설정한 `SimpleImputer` 인스턴스 생성

In [37]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

중간값은 숫자 속성에 대해서만 계산할 수 있으므로, 문자열 속성인 `ocean_proximity`를 제외한 데이터 복사본을 만들어야 함

In [38]:
housing_num = housing.drop("ocean_proximity", axis=1)  
# 또는: housing_num = housing.select_dtypes(include=[np.number]),숫자형 데이터만 선택

이제 `fit()` 메서드를 사용하여 imputer 인스턴스를 훈련 데이터에 학습

In [39]:
imputer.fit(housing_num)

,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False


이 imputer는 각 속성의 중간값(median)을 계산하여 statistics_라는 인스턴스 변수에 저장  
이번 경우에는 total_bedrooms 속성에만 결측값이 있었지만,  
시스템이 운영에 들어간 이후에는 새로운 데이터에도 결측값이 생길 수 있으므로 모든 숫자 속성에 대해 imputer를 적용하는 것이 더 안전

In [40]:
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [41]:
housing_num.median().values

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

이제 이렇게 학습된 imputer를 사용하여 훈련 세트의 누락된 값을 중간값으로 대체

In [1]:
X = imputer.transform(housing_num) # 변환된 특성들이 담긴 NumPy 배열
# imputer 객체를 사용하여 housing_num의 결측치를 대체

NameError: name 'imputer' is not defined

In [5]:
# 다시 pandas DataFrame으로 되돌리기(누락값 처리 → 변환 → 다시 DataFrame으로 복원)
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing.index)

NameError: name 'pd' is not defined

In [48]:
housing_tr.loc[sample_incomplete_rows.index.values]
# sample_incomplete_rows의 인덱스를 사용하여 결측치가 있던 행들만 추출

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


In [52]:
imputer.strategy # 결측치 무엇으로 채우나?

'median'

In [55]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)
# X 데이터를 DataFrame으로 변환, 원본 housing_num의 컬럼명과 인덱스를 그대로 유지
# -> 결측치가 처리된 데이터는 원본 데이터와 같은 구조를 유지

In [51]:
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964


## Handling Text and Categorical Attributes

텍스트 속성도 살펴보기  
이 데이터셋에는 `ocean_proximity` 속성만 텍스트 속성-> 처음 10개 인스턴스 확인

In [56]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


임의의 텍스트가 아니라 가능한 값들이 제한되어 있으며 각각은 하나의 범주를 나타냄 -> **범주형 속성**  
대부분의 머신러닝 알고리즘은 숫자로 작업하는 것을 선호하므로   
이러한 범주들을 텍스트에서 숫자로 변환(Scikit-Learn의 `OrdinalEncoder` 클래스)

In [59]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat) # "ocean_proximity" 컬럼만 포함하는 새로운 데이터프레임
housing_cat_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

카테고리 목록은 `categories_` 인스턴스 변수를 통해 확인  
-> 각 범주형 속성마다 하나씩의 1차원 배열을 포함하는 리스트(범주형 속성이 하나뿐이므로, 하나의 배열만 포함한 리스트가 됨)

In [61]:
ordinal_encoder.categories_ # OrdinalEncoder 객체에서 카테고리형 변수의 각 고유 값 목록을 반환

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

이 표현 방식의 문제: 머신러닝 알고리즘이 서로 가까운 값들이 먼 값들보다 더 유사하다고 가정한다는 것
>일부 경우에는 괜찮을 수도 있음(예: "bad", "average", "good", "excellent"처럼 순서가 있는 범주의 경우)  
하지만 `ocean_proximity` 열에서는 명백히 그렇지 않음(예: 0번과 4번 카테고리가 0번과 1번보다 더 유사함)

이 문제를 해결하기 위해 카테고리마다 하나의 이진 속성을 생성
>예를 들어, 카테고리가 "`<1H OCEAN`"일 때 1이고, 그렇지 않으면 0인 속성을 하나 만들고,   
"`INLAND`"일 때 1인 속성, 그리고 그 외 카테고리들에 대해서도 각각 속성을 만드는 것 -> **원-핫 인코딩(one-hot encoding)**

항상 **하나의 속성만 1(hot)** 이고 나머지는 모두 0(cold)이 된다.

이렇게 새로 만들어진 속성: **더미 속성(dummy attributes)**  
Scikit-Learn은 범주형 값을 원-핫 벡터로 변환해주는 `OneHotEncoder` 클래스 제공

In [62]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 16512 stored elements and shape (16512, 5)>

출력 결과는 NumPy 배열이 아니라 SciPy의 **희소 행렬(sparse matrix)**
> 범주형 속성에 수천 개의 카테고리가 있을 때 매우 유용  
원-핫 인코딩을 수행하면 수천 개의 열이 생긴 행렬이 만들어지는데,  
이 행렬은 각 행마다 단 하나의 1을 제외하고는 전부 0으로 채워짐  
이러한 0들을 저장하기 위해 엄청난 메모리를 사용하는 것은 매우 비효율적이므로  
희소 행렬은 0이 아닌 원소의 위치만 저장함

희소 행렬은 대부분의 경우 일반적인 2차원 배열처럼 사용할 수 있지만,  
정말로 밀집(dense)한 NumPy 배열로 변환하고 싶다면 `toarray()` 메서드 호출

In [63]:
housing_cat_1hot.toarray()
# 기본적으로 OneHotEncoder 클래스는 희소 배열을 반환하지만,  
# 필요한 경우 toarray() 메서드를 호출하여 이를 밀집 배열로 변환할 수 있음

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], shape=(16512, 5))

In [67]:
# OneHotEncoder를 생성할 때 sparse_output=False를 설정할 수 있다
cat_encoder = OneHotEncoder(sparse_output=False) # 예전에는 sparse=False 사용!
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], shape=(16512, 5))

인코더의 `categories_` 인스턴스 변수를 사용하여 카테고리 목록을 확인

In [68]:
cat_encoder.categories_


[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

범주형 속성에 가능한 카테고리의 수가 많을 경우(예: 국가 코드, 직업, 종 등), 원-핫 인코딩은 많은 수의 입력 특성으로 이어짐  
-> 학습 속도를 늦추고 성능 저하 

이런 상황이 발생하면, 범주형 입력을 해당 카테고리와 관련된 유용한 수치형 특성으로 대체하는 것이 좋음  
>예를 들어, `ocean_proximity` 속성을 바다까지의 거리로 대체할 수 있음(국가 코드는 그 나라의 인구나 1인당 GDP로 대체 가능)

또는 각 카테고리를 **임베딩(embedding)** 이라고 불리는 학습 가능한 저차원 벡터로 대체할 수도 있음  
각 카테고리의 표현은 학습 과정에서 자동으로 학습됨  
이것은 **표현 학습(representation learning)** 의 한 예

## Custom Transformers(사용자 정의 변환기)

Scikit-Learn은 많은 유용한 변환기를 제공하지만, 사용자 정의 정제 작업이나 특정 속성을 결합하는 작업처럼 직접 작성해야 할 경우도 있음  
이런 변환기를 Scikit-Learn의 기능(예: 파이프라인)과 원활하게 작동하도록 하기 위해  
Scikit-Learn은 상속이 아닌 **덕 타이핑(duck typing)** 기반이므로   
단순히 클래스를 만들고 세 가지 메서드를 구현하면 됨
 >`fit()` (self를 반환)  
`transform()`  
`fit_transform()`

마지막 메서드인 `fit_transform()`은 `TransformerMixin`을 상속하면 자동으로 제공된다.

또한 `BaseEstimator`를 상속하고 생성자에 `*args`나 `**kargs`를 사용하지 않으면,  
자동 하이퍼파라미터 튜닝에 유용한 두 가지 메서드(`get_params()`와 `set_params()`)도 사용할 수 있게 됨


In [93]:
import pickle
from sklearn.base import BaseEstimator, TransformerMixin  # scikit-learn에서 BaseEstimator와 TransformerMixin을 임포트

# 각 컬럼의 인덱스를 변수로 설정
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):  # 생성자에서 add_bedrooms_per_room 인자를 받음
        self.add_bedrooms_per_room = add_bedrooms_per_room  # 기본값 True로 설정

    def fit(self, X, y=None):
        return self  # fit 메서드는 변환할 데이터가 없으므로 그대로 반환

    def transform(self, X):
        # 각 특성들을 계산하여 새로운 특성 추가
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]  # 가구당 방 수
        population_per_household = X[:, population_ix] / X[:, households_ix]  # 가구당 인구 수
        if self.add_bedrooms_per_room:
            # add_bedrooms_per_room이 True일 경우, 추가 특성: 방당 침실 수
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]  
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]  # 새로운 특성 포함하여 반환
        else:
            return np.c_[X, rooms_per_household, population_per_household]  # 추가 특성만 포함하여 반환

# CombinedAttributesAdder 클래스를 이용해 인스턴스를 만들고, add_bedrooms_per_room을 False로 설정
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)

# housing 데이터를 변환하여 새로운 특성을 추가한 결과를 housing_extra_attribs에 저장
housing_extra_attribs = attr_adder.transform(housing.values)

# pickle로 CombinedAttributesAdder 객체 저장(다른 노트북에서 쓰기 위함)
with open('combined_attributes_adder.pkl', 'wb') as f:
    pickle.dump(attr_adder, f)


In [74]:
# 간결성과 명확성을 위해 인덱스(3, 4, 5, 6)를 하드코딩했지만, 
# 다음과 같이 동적으로 가져오는 것이 훨씬 더 깔끔할 것이다
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names] # get the column indices

In [75]:
# housing_extra_attribs는 NumPy 배열이므로 열 이름이 손실(Scikit-Learn 문제)
# DataFrame 복구
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND,5.485836,3.168555
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN,6.927083,2.623698
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.875,INLAND,5.393333,2.223333
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN,3.886128,1.859213
4,-118.7,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN,6.096552,3.167241


변환기는 하나의 하이퍼파라미터 `add_bedrooms_per_room`을 가지고 있으며, 기본값은 `True`  
이 하이퍼파라미터를 통해 이 속성(`bedrooms_per_room`)을 추가하는 것이 머신러닝 알고리즘의 성능 향상에 도움이 되는지 실험해 볼 수 있음

100% 확신이 없는 데이터 준비 단계에 대해 하이퍼파라미터를 추가해 두면 유용  
이런 데이터 준비 작업을 자동화하면 할수록 다양한 조합을 자동으로 실험해볼 수 있게 되어,   
더 좋은 조합을 발견할 가능성이 높아지고 많은 시간을 절약할 수 있기 때문

## Feature Scaling(특성 스케일링)

데이터에 적용해야 하는 가장 중요한 변환 중 하나

몇 가지 예외를 제외하면, 머신러닝 알고리즘은 입력 숫자 속성들의 **스케일이 매우 다를 경우** 제대로 작동하지 않음(주택 데이터 해당)  
총 방 개수는 대략 6개에서 39,320개까지 다양하지만 중간 소득은 0에서 15 사이에 불과
일반적으로 타깃 값은 스케일링할 필요가 없음  

모든 속성의 스케일을 같게 만드는 일반적인 방법: 최소-최대 스케일링, 표준화

1. 최소-최대 스케일링(min-max scaling, 정규화(normalization))은 가장 간단한 방법  
값들을 이동시키고 재조정하여 0에서 1 사이에 오도록 함  
최소값을 빼고, (최댓값 - 최소값)으로 나누는 방식  
Scikit-Learn은 이를 위한 `MinMaxScaler`라는 변환기를 제공  
이 클래스에는 `feature_range`라는 하이퍼파라미터가 있어, 필요에 따라 0–1 이외의 범위를 지정할 수도 있음

2. 표준화
먼저 평균값을 빼므로 표준화된 값들은 항상 평균이 0이 됨  
그리고 나서 표준편차로 나누어 분산이 1이 되도록 만듦  
최소-최대 스케일링과 달리 표준화는 값을 특정 범위로 제한하지 않으며 이는 일부 알고리즘(예: 0-1 사이의 입력을 기대하는 신경망)에는 문제가 될 수 있음   
하지만 표준화는 이상치(outlier)의 영향을 훨씬 덜 받음  
>예를 들어, 어떤 구역의 중간 소득이 실수로 100이라고 하면 최소-최대 스케일링은 나머지 015의 값을 0~0.15 범위로 눌러버릴 것이지만,  
표준화는 큰 영향 X -> Scikit-Learn은 표준화를 위한 StandardScaler라는 변환기 제공

모든 변환과 마찬가지로, 스케일러는 전체 데이터셋(테스트 세트를 포함한)이 아니라 훈련 데이터에만 맞춰서(fit) 학습시키는 것이 중요함  
그렇게 해야만 훈련 세트와 테스트 세트(그리고 새로운 데이터)를 변환하는 데 해당 스케일러를 사용할 수 있음!

## Transformation Pipelines(변환 파이프라인)

In [76]:
# pipeline 빌드
from sklearn.pipeline import Pipeline  # scikit-learn에서 Pipeline을 임포트
from sklearn.preprocessing import StandardScaler  # StandardScaler를 임포트

# 숫자형 데이터를 위한 파이프라인 정의
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),  # 결측치는 중앙값으로 대체
        ('attribs_adder', CombinedAttributesAdder()),  # 새로운 특성(가구당 방 수, 인구 수 등) 추가
        ('std_scaler', StandardScaler()),  # 데이터 표준화 (평균 0, 분산 1)
    ])

# housing_num 데이터에 대해 파이프라인을 적용하여 변환된 데이터 얻기
housing_num_tr = num_pipeline.fit_transform(housing_num)


In [77]:
housing_num_tr

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.01739526,
         0.00622264, -0.12112176],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.56925554,
        -0.04081077, -0.81086696],
       [ 0.26758118, -0.1259716 ,  1.22045984, ..., -0.01802432,
        -0.07537122, -0.33827252],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ..., -0.5092404 ,
        -0.03743619,  0.32286937],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.32814891,
        -0.05915604, -0.45702273],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.01407228,
         0.00657083, -0.12169672]], shape=(16512, 11))

`Pipeline` 생성자는 단계들의 순서를 정의하는 이름/추정기 쌍의 리스트를 받음  
마지막 추정기를 제외한 모든 추정기는 변환기여야 함(즉, `fit_transform()` 메서드를 가지고 있어야 함)   
이름은 무엇이든 상관없지만, 고유해야 하고 이중 밑줄(`__`)을 포함해서는 안 됨  
이 이름들은 하이퍼파라미터 튜닝 시 유용하게 사용됨 

파이프라인의 `fit()` 메서드를 호출하면  
모든 변환기에 대해 `fit_transform()`을 순차적으로 호출하면서 각 호출의 출력을 다음 호출의 입력으로 전달함  
마지막 추정기에 도달하면 `fit()` 메서드를 호출  

파이프라인은 마지막 추정기와 동일한 메서드들 제공  
이 예제에서 마지막 추정기는 변환기인 `StandardScaler`이므로   
파이프라인은 데이터를 순서대로 변환하는 `transform()` 메서드를 가지고 있으며  
`fit_transform()` 메서드도 물론 포함되어 있음  

지금까지 범주형 열과 숫자형 열을 따로 처리했으나  
모든 열을 적절하게 변환하는 단일 변환기가 있다면 더 편리할 것  
Scikit-Learn의 `ColumnTransformer`는 pandas DataFrame과 잘 작동함  

이제 이를 사용하여 주택 데이터를 모두 변환

In [78]:
from sklearn.compose import ColumnTransformer  # ColumnTransformer를 임포트

# 숫자형 및 범주형 속성 리스트 정의
num_attribs = list(housing_num)  # 숫자형 속성 목록 (housing_num의 모든 컬럼 이름)
cat_attribs = ["ocean_proximity"]  # 범주형 속성 목록

# 전체 파이프라인 정의
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),  # 숫자형 데이터는 num_pipeline을 적용
        ("cat", OneHotEncoder(), cat_attribs),  # 범주형 데이터는 OneHotEncoder를 적용
    ])

# housing 데이터에 대해 전체 파이프라인을 적용하여 변환된 데이터 얻기
housing_prepared = full_pipeline.fit_transform(housing)


In [79]:
housing_prepared

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ...,  0.        ,
         0.        ,  0.        ],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.        ,
         0.        ,  0.        ]], shape=(16512, 16))

먼저 `ColumnTransformer` 클래스를 임포트하고, 
숫자형 열 이름들과 범주형 열 이름들의 리스트를 가져온 다음 `ColumnTransformer` 구성
이 생성자는 튜플들의 리스트를 필요로 함
>1. 이름
>2. 변환기: 적용할 전처리기(예: `StandardScaler()`, `OneHotEncoder()` 등)
>3. 변환기를 적용할 열들의 이름(또는 인덱스) 리스트 포함

여기에서는 앞서 정의한 `num_pipeline`을 사용해 숫자형 열을 변환하고, `OneHotEncoder`를 사용해 범주형 열을 변환하도록 지정  
마지막으로 이 `ColumnTransformer`를 주택 데이터에 적용  
-> 각각의 변환기가 해당 열에 적용되고, 출력은 두 번째 축을 따라 연결(변환기는 동일한 행 수 반환)

주의) `OneHotEncoder`는 희소 행렬을 반환하는 반면, `num_pipeline`은 밀집 행렬을 반환  
>희소 행렬과 밀집 행렬이 혼합되어 있을 경우 `ColumnTransformer`는 최종 행렬의 밀도를 추정(즉, 0이 아닌 셀의 비율)  
그리고 그 밀도가 특정 임계값보다 낮으면 희소 행렬을 반환(기본값은 `sparse_threshold=0.3`)  
이 예제에서는 밀집 행렬을 반환

=> 전체 주택 데이터를 입력으로 받아 각 열에 적절한 변환을 적용하는 전처리 파이프라인 완성

변환기를 사용하는 대신 열을 삭제 -> 문자열 `drop` 지정 가능  
열을 그대로 두기 -> `pass through` 지정 
>기본적으로 나머지 열들(즉, 명시되지 않은 열들)은 삭제되지만  
이러한 열들을 다르게 처리하고 싶으면 `remainder` 하이퍼파라미터를 임의의 변환기나 `passthrough`로 설정할 수 있음

In [80]:
housing_prepared.shape

(16512, 16)

In [81]:
# DataFrameSelector 변환기(Pandas DataFrame 열의 하위 집합만 선택)와 
# FeatureUnion을 기반으로 한 기존 솔루션
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
class OldDataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [85]:
# 모든 구성 요소를 숫자형 및 범주형 기능을 모두 전처리하는 큰 파이프라인으로 결합

# 숫자형 및 범주형 속성 리스트 정의
num_attribs = list(housing_num)  # 숫자형 속성 목록 (housing_num의 모든 컬럼 이름)
cat_attribs = ["ocean_proximity"]  # 범주형 속성 목록

# 숫자형 데이터 처리 파이프라인 정의
old_num_pipeline = Pipeline([
        ('selector', OldDataFrameSelector(num_attribs)),  # 숫자형 속성 선택 (OldDataFrameSelector는 사용자 정의 클래스일 가능성 있음)
        ('imputer', SimpleImputer(strategy="median")),  # 결측치를 중앙값으로 대체
        ('attribs_adder', CombinedAttributesAdder()),  # 새로운 특성(가구당 방 수, 인구 수 등) 추가
        ('std_scaler', StandardScaler()),  # 데이터 표준화 (평균 0, 분산 1)
    ])

# 범주형 데이터 처리 파이프라인 정의
old_cat_pipeline = Pipeline([
        ('selector', OldDataFrameSelector(cat_attribs)),  # 범주형 속성 선택 (OldDataFrameSelector는 사용자 정의 클래스일 가능성 있음)
        ('cat_encoder', OneHotEncoder(sparse_output=False)),  # 범주형 데이터는 OneHotEncoder로 변환, sparse_output=False는 희소 행렬 대신 밀집 행렬 반환
    ])


In [86]:
from sklearn.pipeline import FeatureUnion  # FeatureUnion을 임포트

# 숫자형 데이터 처리 파이프라인과 범주형 데이터 처리 파이프라인을 합치는 FeatureUnion 정의
old_full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", old_num_pipeline),  # 숫자형 데이터 처리 파이프라인을 "num_pipeline"으로 추가
        ("cat_pipeline", old_cat_pipeline),  # 범주형 데이터 처리 파이프라인을 "cat_pipeline"으로 추가
    ])

In [87]:
# 전체 파이프라인을 사용하여 housing 데이터를 변환하고, 변환된 데이터를 old_housing_prepared에 저장
old_housing_prepared = old_full_pipeline.fit_transform(housing)

# 변환된 데이터 출력 (변환된 housing 데이터 확인)
old_housing_prepared

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ...,  0.        ,
         0.        ,  0.        ],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.        ,
         0.        ,  0.        ],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.        ,
         0.        ,  0.        ]], shape=(16512, 16))

In [89]:
# 결과는 ColumnTransformer와 동일
np.allclose(housing_prepared, old_housing_prepared)

True

In [90]:
import pickle

# full_pipeline을 pickle로 저장
with open('full_pipeline.pkl', 'wb') as f:
    pickle.dump(full_pipeline, f)


In [38]:
import joblib
housing_prepared = joblib.load("data/housing_prepared.pkl")
housing_labels = joblib.load("data/housing_labels.pkl")
full_pipeline = joblib.load("models/full_pipeline.pkl")

In [13]:
from sklearn.linear_model import LinearRegression  # 선형 회귀 모델 임포트

lin_reg = LinearRegression()  # 모델 객체 생성
lin_reg.fit(housing_prepared, housing_labels)  # 훈련 데이터로 모델 학습

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [14]:
# 예측 테스트 (5개 샘플)

some_data = housing.iloc[:5]  # 원본 데이터에서 일부 샘플 추출
some_labels = housing_labels.iloc[:5]  # 실제 값 추출
some_data_prepared = full_pipeline.transform(some_data)  # 전처리 적용

print("Predictions:", lin_reg.predict(some_data_prepared))  # 예측값 출력
print("Labels:", list(some_labels))  # 실제값 출력

Predictions: [ 85657.90192014 305492.60737488 152056.46122456 186095.70946094
 244550.67966089]
Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [15]:
# RMSE 계산 (훈련 세트 기준)
from sklearn.metrics import mean_squared_error  # RMSE 계산용 함수

housing_predictions = lin_reg.predict(housing_prepared)  # 전체 예측
lin_mse = mean_squared_error(housing_labels, housing_predictions)  # MSE 계산
lin_rmse = np.sqrt(lin_mse)  # RMSE 계산

In [16]:
# 결정 트리 모델 훈련
from sklearn.tree import DecisionTreeRegressor  

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)  # 학습

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [17]:
# RMSE 계산 (훈련 세트 기준, 과적합 여부 확인)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)  # 훈련 데이터에 대해 RMSE가 0일 수도 있음 → 과적합 신호

In [18]:
# 교차 검증 (결정 트리)
from sklearn.model_selection import cross_val_score  # 교차 검증 함수

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)  # 10겹 교차검증
rmse_scores = np.sqrt(-scores)  # 점수가 음수이므로 반전 후 제곱근 계산

In [19]:
# 점수 출력 함수
def display_scores(scores):  # 점수 리스트를 보기 좋게 출력
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [72501.64212996 71513.40218484 67830.54919907 71883.14883739
 69018.2726938  76111.58157257 71451.05987091 73682.5472033
 66976.23403519 70848.44973403]
Mean: 71181.68874610616
Standard deviation: 2582.2946569025576


In [20]:
# 교차 검증 (선형 회귀 비교)
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
Standard deviation: 2880.3282098180666


In [21]:
# 랜덤 포레스트 훈련 및 평가
from sklearn.ensemble import RandomForestRegressor  # 랜덤 포레스트 임포트

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)  # 학습

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [51314.22213553 48933.25747258 46921.87189441 51649.30898705
 47600.76141668 51553.51720848 52409.02769873 49429.45814488
 48572.32898066 53861.81812521]
Mean: 50224.557206421174
Standard deviation: 2140.2460267705574


In [22]:
# 모델 저장 및 재사용
import joblib  # 대용량 모델 저장에 적합한 라이브러리

joblib.dump(forest_reg, "forest_model.pkl")  # 모델 저장
model_loaded = joblib.load("forest_model.pkl")  # 저장된 모델 불러오기


In [35]:
joblib.dump(lin_reg, "models/lin_reg.pkl")

['models/lin_reg.pkl']

In [36]:
import joblib
housing_prepared = joblib.load("data/housing_prepared.pkl")
housing_labels = joblib.load("data/housing_labels.pkl")

In [31]:

from sklearn.model_selection import GridSearchCV  # 하이퍼파라미터 튜닝을 위한 GridSearchCV 임포트
from sklearn.ensemble import RandomForestRegressor  # 랜덤 포레스트 회귀 모델 임포트

param_grid = [  # 탐색할 하이퍼파라미터 조합 정의
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()  # 모델 객체 생성

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,  # 그리드 탐색 객체 생성
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)  # 전체 훈련 세트로 학습 및 교차 검증

,estimator,RandomForestRegressor()
,param_grid,"[{'max_features': [2, 4, ...], 'n_estimators': [3, 10, ...]}, {'bootstrap': [False], 'max_features': [2, 3, ...], 'n_estimators': [3, 10]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,n_estimators,30


In [24]:
grid_search.best_params_  # 가장 좋은 하이퍼파라미터 조합 출력

{'max_features': 8, 'n_estimators': 30}

In [25]:
grid_search.best_estimator_  # 가장 성능이 좋은 모델 출력

,n_estimators,30
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,8
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [26]:
cvres = grid_search.cv_results_  # 교차 검증 결과 저장
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)  # RMSE와 하이퍼파라미터 조합 출력

64694.09167288166 {'max_features': 2, 'n_estimators': 3}
55164.88624325165 {'max_features': 2, 'n_estimators': 10}
52551.38765595925 {'max_features': 2, 'n_estimators': 30}
60388.731146036764 {'max_features': 4, 'n_estimators': 3}
53267.37802899572 {'max_features': 4, 'n_estimators': 10}
50207.50084450273 {'max_features': 4, 'n_estimators': 30}
59805.319690099546 {'max_features': 6, 'n_estimators': 3}
52377.60068584532 {'max_features': 6, 'n_estimators': 10}
50125.15542823475 {'max_features': 6, 'n_estimators': 30}
57755.02333209064 {'max_features': 8, 'n_estimators': 3}
52331.65182011444 {'max_features': 8, 'n_estimators': 10}
50109.42823492966 {'max_features': 8, 'n_estimators': 30}
62191.866890750294 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54309.763829849544 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60014.37816524238 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52903.33724230673 {'bootstrap': False, 'max_features': 3, 'n_estimators'

In [27]:
feature_importances = grid_search.best_estimator_.feature_importances_  # 특성 중요도 추출

In [28]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]  # 파생 변수 이름
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out())  # 원-핫 인코딩된 범주형 변수 이름
attributes = num_attribs + extra_attribs + cat_one_hot_attribs  # 전체 특성 이름 합치기

In [29]:
sorted(zip(feature_importances, attributes), reverse=True)  # 특성 중요도 내림차순 정렬

[(np.float64(0.34213281602836687), 'median_income'),
 (np.float64(0.15558810299896655), 'ocean_proximity_INLAND'),
 (np.float64(0.11607402344039748), 'pop_per_hhold'),
 (np.float64(0.07590574708074133), 'bedrooms_per_room'),
 (np.float64(0.06727026530557186), 'longitude'),
 (np.float64(0.06477306726350168), 'latitude'),
 (np.float64(0.05902832413253498), 'rooms_per_hhold'),
 (np.float64(0.0429404779582283), 'housing_median_age'),
 (np.float64(0.01534387031122192), 'total_rooms'),
 (np.float64(0.014825442172739667), 'total_bedrooms'),
 (np.float64(0.014780045886326495), 'population'),
 (np.float64(0.014494803040373677), 'households'),
 (np.float64(0.011120000400319432), 'ocean_proximity_<1H OCEAN'),
 (np.float64(0.0029494573560954864), 'ocean_proximity_NEAR OCEAN'),
 (np.float64(0.002710684510062972), 'ocean_proximity_NEAR BAY'),
 (np.float64(6.287211455126793e-05), 'ocean_proximity_ISLAND')]

In [30]:
from sklearn.metrics import mean_squared_error  # RMSE 계산을 위한 모듈 임포트

final_model = grid_search.best_estimator_  # 최종 모델 지정

X_test = strat_test_set.drop("median_house_value", axis=1)  # 테스트 입력 특성
y_test = strat_test_set["median_house_value"].copy()  # 테스트 타깃 레이블

X_test_prepared = full_pipeline.transform(X_test)  # 전처리 파이프라인 적용
final_predictions = final_model.predict(X_test_prepared)  # 예측 수행

final_mse = mean_squared_error(y_test, final_predictions)  # MSE 계산
final_rmse = np.sqrt(final_mse)  # RMSE 계산
print(final_rmse)  # RMSE 출력

47800.95305004226


In [37]:
joblib.dump(grid_search, "models/grid_search.pkl")

['models/grid_search.pkl']

In [95]:
import pickle

# housing_prepared 데이터를 pickle로 저장
with open('housing_prepared.pkl', 'wb') as f:
    pickle.dump(housing_prepared, f)
